In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

import config as cfg

In [2]:
# Download stock prices
stock_data = yf.download(cfg.tickers, start=cfg.start_date, end=cfg.end_date)

# Select 'Close' prices for each stock
close_prices = stock_data['Close']

# Transform stock prices to a dataframe
df = pd.DataFrame(close_prices)
print(df.head(10))

[*********************100%***********************]  212 of 212 completed

14 Failed downloads:
- 2CRSI.PA: No data found, symbol may be delisted
- TIS.MI: No data found, symbol may be delisted
- AKA.PA: No data found, symbol may be delisted
- HOLN.PA: No data found, symbol may be delisted
- EUCAR.PA: No data found, symbol may be delisted
- HEXA.PA: No data found, symbol may be delisted
- ATA.PA: No data found, symbol may be delisted
- CNP.PA: No data found, symbol may be delisted
- STM.PA: No data found, symbol may be delisted
- URW.AS: No data found, symbol may be delisted
- ATI.PA: No data found, symbol may be delisted
- ABIO.PA: No data found, symbol may be delisted
- GOE.PA: No data found, symbol may be delisted
- PHA.PA: No data found, symbol may be delisted
                     2CRSI.PA  A2A.MI  AB.PA  ABCA.PA  ABIO.PA  ABNX.PA  \
Date                                                                      
2012-01-04 00:00:00       NaN  0.7575   5.60     6.18      NaN      NaN   
2

In [12]:
# watch_days_range = [5,10,20,30,60]
# hold_days_days_range = [5,10,30,100,200,300,400]
# num_stocks_to_buy_range = [5,10,20,30]
# loss_limit_range = [1,0.98,0.9,0.5,0.2]

watch_days_range = [5,30]
hold_days_days_range = [5,30]
num_stocks_to_buy_range = [1]
loss_limit_range = [0.99,0.9]

num_combinations = len(watch_days_range) * len(hold_days_days_range) * len(num_stocks_to_buy_range) * len(loss_limit_range)
combinations_per_minute = 49

print(f"Number of cases: {num_combinations}, Estimated time: {num_combinations / combinations_per_minute} minutes")

Number of cases: 8, Estimated time: 0.16326530612244897 minutes


In [17]:
results = []
combination_index = 0  # Index for results array

# Repeat for each period of watch_days
for watch_days in watch_days_range:  # watch_days represents the number of days for price increase calculation
    # num_iterations = (len(df) - watch_days + 1) // watch_days # Calculate the number of iterations
    for hold_days in hold_days_days_range:  # hold_days represents the number of additional days before selling. Repeat for each period of watch_days plus hold_days
        num_iterations = int((len(df) - hold_days) / watch_days) - 1

        for num_stocks_to_buy in num_stocks_to_buy_range:  # num_stocks_to_buy represents the number of stocks to buy
            for loss_limit in loss_limit_range:  # loss_limit represents the percentage decrease threshold for selling
                total_profit = 1

                for i in range(num_iterations):
                    start_watch_day_number = i * watch_days
                    buy_day_number = start_watch_day_number + watch_days
                    sell_day_number = buy_day_number + hold_days

                    price_increase = df[start_watch_day_number:buy_day_number].pct_change(watch_days - 1).tail(1) # Calculate the price increase in the last watch_days days for each stock
                    price_increase.dropna(axis=1) # remove columns with n/a values

                    top_stocks = price_increase.squeeze().nlargest(num_stocks_to_buy).index # Select the num_stocks_to_buy stocks with the highest price increase

                    buy_prices = df.loc[df.index[buy_day_number], top_stocks] # Calculate the buying prices at the start of the hold period
                    sell_prices = df.loc[df.index[sell_day_number], top_stocks] # Calculate the selling prices after watch_days + hold_days days

                    profits = sell_prices / buy_prices # Calculate the profit for each stock

                    sell_mask = sell_prices / buy_prices < loss_limit # Filter the stocks to sell based on decrease threshold loss_limit
                    profits[sell_mask] = loss_limit  # Set profit to loss_limit for stocks that decrease beyond threshold

                    total_profit *= profits.mean(skipna=True) # Calculate the total profit for the selected stocks (i.e. average of profits)
                    print(f"buy_prices \n {buy_prices} \n sell_prices \n {sell_prices} \n profits {profits} \
                          \n total_profit: {total_profit}")

                results.append({'watch_days': watch_days, 'num_stocks_to_buy': num_stocks_to_buy, 'hold_days': hold_days,
                                'loss_limit': loss_limit, 'total_profit': total_profit})
                
                print(f"index: {combination_index}")
                combination_index += 1

results_df = pd.DataFrame(results)

buy_prices 
 VLTSA.PA    10.848149
Name: 2012-01-11 00:00:00, dtype: float64 
 sell_prices 
 VLTSA.PA    13.057957
Name: 2012-01-18 00:00:00, dtype: float64 
 profits VLTSA.PA    1.203704
dtype: float64
buy_prices 
 METEX.PA    4.87
Name: 2012-01-18 00:00:00, dtype: float64 
 sell_prices 
 METEX.PA    4.7
Name: 2012-01-25 00:00:00, dtype: float64 
 profits METEX.PA    0.99
dtype: float64
buy_prices 
 UCG.MI    19.039248
Name: 2012-01-25 00:00:00, dtype: float64 
 sell_prices 
 UCG.MI    19.901024
Name: 2012-02-01 00:00:00, dtype: float64 
 profits UCG.MI    1.045263
dtype: float64
buy_prices 
 COX.PA    7.45
Name: 2012-02-01 00:00:00, dtype: float64 
 sell_prices 
 COX.PA    9.15
Name: 2012-02-08 00:00:00, dtype: float64 
 profits COX.PA    1.228188
dtype: float64
buy_prices 
 AB.PA    12.66
Name: 2012-02-08 00:00:00, dtype: float64 
 sell_prices 
 AB.PA    11.54
Name: 2012-02-15 00:00:00, dtype: float64 
 profits AB.PA    0.99
dtype: float64
buy_prices 
 AKW.PA    4.183
Name: 2012-02-

KeyboardInterrupt: 

In [14]:
print(results_df)

   watch_days  num_stocks_to_buy  hold_days  loss_limit  total_profit
0           5                  1          5        0.99           NaN
1           5                  1          5        0.90           NaN
2           5                  1         30        0.99           NaN
3           5                  1         30        0.90           NaN
4          30                  1          5        0.99     32.525569
5          30                  1          5        0.90      6.742662
6          30                  1         30        0.99   1830.456112
7          30                  1         30        0.90     46.597667


In [15]:
results_df.sort_values("total_profit", ascending=False, inplace=True)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(results_df)

   watch_days  num_stocks_to_buy  hold_days  loss_limit  total_profit
6          30                  1         30        0.99      1830.456
7          30                  1         30        0.90        46.598
4          30                  1          5        0.99        32.526
5          30                  1          5        0.90         6.743
0           5                  1          5        0.99           NaN
1           5                  1          5        0.90           NaN
2           5                  1         30        0.99           NaN
3           5                  1         30        0.90           NaN


In [16]:
from datetime import datetime

def get_number_of_years(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    delta = end - start
    return delta.days / 365.25

top_profit = results_df['total_profit'].max()
years = get_number_of_years(cfg.start_date, cfg.end_date)
top_yearly_profit = pow(top_profit, 1 / years)

print(f"years: {years:.2f}, top_profit: x{top_profit:.2f}, top_yearly_profit: x{top_yearly_profit:.2f}")

years: 10.99, top_profit: x1830.46, top_yearly_profit: x1.98
